In [ ]:
import torch
import numpy as np
import pandas as pd
import pickle
torch.cuda.empty_cache()

In [ ]:
!pip install -U transformers
!pip install -U torch

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1IG7APAMDUOBfToE_NDM9uKczkIUNC5__d9BsEK1DMIY/edit#gid=1880462315'
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
df = pd.read_csv(url_1, header=None)
df

,0,1
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


### Language Augmentation

In [ ]:
import torch

from transformers import pipeline

def augment_data(texts, device=0):
    # Load the translation pipelines for English to French and French to English
    en_to_fr = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr", device=device)
    fr_to_en = pipeline("translation_fr_to_en", model="Helsinki-NLP/opus-mt-fr-en", device=device)

    # Translate the input English text to French
    french_texts = en_to_fr(texts, max_length=315)

    # Extract the translated French text
    french_texts = [text["translation_text"] for text in french_texts]

    # Translate the French text back to English
    backtranslated_texts = fr_to_en(french_texts, max_length=315)

    # Extract the backtranslated English text
    backtranslated_texts = [text["translation_text"] for text in backtranslated_texts]

    return backtranslated_texts



# text = "According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
# augmented_text = augment_data(text)
# print(augmented_text)

In [ ]:
!pip install sacremoses

In [ ]:
import torch
texts_aug = []
labels_aug = []

texts_aug = augment_data(X_train.tolist())

with open("/content/drive/MyDrive/CSC413/Project/aug_data.pkl", "wb") as f:
    pickle.dump([texts_aug, t_train], f)

In [ ]:
texts = np.concatenate([texts, texts_aug])
labels = np.concatenate([labels, labels_aug])

In [ ]:
print('from non-augmentated dataset:')
print(texts[1])
print(labels[1])
print('from augmentated dataset:')
print(texts[4847])
print(labels[4847])